In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !ls '/content/drive/My Drive/Datamining/data'

In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
file_name = '../../data/fd-export.csv'

## read file
df = pd.read_csv(file_name)
cond_file_name = '../../data/conditions_list.csv'
df_conditions = pd.read_csv(cond_file_name)
"""
    Function to remove abnormal age.
    This is required for reusability purpose.
    
    @Input : dataframe
    @Return : dataframe
"""
def remove_abnormal_age(df):
    df.age = df.age.fillna(-1)
    invalid_ids = set(df[ (df.age<0) | (df.age > 100) ].user_id.values)
    valid_df = df[~df.user_id.isin(invalid_ids)]
    
    print("Valid users with norma age = {}, Percentage {}".format( valid_df.user_id.unique().shape[0]
                                                                  , valid_df.user_id.unique().shape[0]/
                                                               float( df.user_id.unique().shape[0] ) ))
    
    return valid_df

## filter the user
df_processed = remove_abnormal_age(df)

df_processed.head(10)
    

/home/shibintv/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Valid users with norma age = 20456, Percentage 0.9268270581305786


,user_id,age,sex,country,checkin_date,trackable_id,trackable_type,trackable_name,trackable_value
1,QEVuQwEAWRNGnuTRqXG2996KSkTIEw==,31.0,male,US,2015-11-26,1069,Condition,Ulcerative colitis,0
2,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3168,Condition,pain in left upper arm felt like i was getting...,4
3,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3169,Condition,hip pain when gettin up,3
4,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3170,Condition,pain in hand joints,4
5,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3171,Condition,numbness in right hand,2
6,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,1356,Condition,Headache,2
7,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3172,Condition,pain in left ankle,1
8,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3173,Condition,pain in left leg,1
9,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3174,Condition,pain in joints on feet,2
10,QEVuQwEA+WkNxtp/qkHvN2YmTBBDqg==,1.0,female,CA,2017-04-28,3175,Condition,neck and upper back pain,2


In [3]:


def filter_the_symptoms(df):
  symptoms = df[(df['trackable_type'] == "Symptom")].trackable_id.value_counts().sort_values(ascending = False)[:5000].index
  print(symptoms[0])
  df = df[df.trackable_id.isin(symptoms)]
  return df
df = filter_the_symptoms(df)

242


# Prediction Task Below

In [32]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

df['checkin_date'] = pd.to_datetime(df['checkin_date'])

def combineConditions(x):
    return set(x)

def makeList(x):
    return list(x)

def numericOr(x):
    if 1 in x.values:
        return 1
    else:
        return 0
    
def reshapeSymptoms(df):
    #reshape and one-hot the symptoms

    symptoms = pd.get_dummies(df[(df['trackable_type'] == "Symptom") & (df['trackable_value'] != 0)], columns=['trackable_name'])
    symptoms = symptoms.drop(['trackable_id', 'trackable_type', 'trackable_value'], axis=1)
    symptoms = symptoms.groupby(['user_id', 'checkin_date']).agg(numericOr).reset_index()
    return symptoms
    
def create_condition_df(df, symptoms):
    newdf = df[df['trackable_type'] == 'Condition']
    newdf = newdf.merge(symptoms, on=['user_id','checkin_date'])

    #newdf = newdf.drop_duplicates().drop(['user_id','checkin_date','trackable_id','trackable_type', 'trackable_value'], axis=1)
    newdf = newdf.drop(['user_id','checkin_date','age_x', 'age_y','sex_y', 'country_y'], axis=1)
    newdf = newdf.groupby(['trackable_name']).agg(sum).reset_index()
    trackable_names = newdf.trackable_name.values.tolist()
    newdf = newdf.drop(['trackable_name'], axis = 1)
    return newdf, trackable_names

In [33]:
df = df[:1000]
symptoms = reshapeSymptoms(df)
symptoms.head(10)
# print(list(symptoms))
condition_df, trackable_names = create_condition_df(df, symptoms)
condition_df.head()

,trackable_name_Abdominal pain,trackable_name_Afternoon fatigue,trackable_name_Allergy,trackable_name_Altered mental status,trackable_name_Anger,trackable_name_Anhedonia,trackable_name_Anxiety,trackable_name_Back pain,trackable_name_Bloating,trackable_name_Body aching,...,trackable_name_heart palpitations,trackable_name_intrusive thoughts,trackable_name_lightheadedness,trackable_name_mania,trackable_name_ovarian pain,trackable_name_panic attacks,trackable_name_sleep drunk,trackable_name_sleepiness,trackable_name_swelling,trackable_name_weepiness
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## Do PCA

In [34]:

from sklearn.decomposition import PCA
import os
n_components = 20
pca = PCA(n_components = n_components)
pca_transform = pca.fit_transform(condition_df.values)




In [43]:
from sklearn.metrics.pairwise import euclidean_distances 
pair_dist = euclidean_distances(pca_transform, pca_transform)

all_pairs = [ (i,j) for i in range(pair_dist.shape[0]) for j in range( i+1, pair_dist.shape[0]) ]

all_pairs = sorted(all_pairs, key = lambda x: pair_dist[x[0]][x[1]] )

def print_similar( all_pairs, conditions, n = 10):
    for i in range(n):
        x = all_pairs[i][0]
        y = all_pairs[i][1]
        print(' Pairs = {} : {} '.format(conditions[x], conditions[y]))
        
print_similar(all_pairs,trackable_names )
        
    


 Pairs = Adult ADHD : Scoliosis 
 Pairs = Ankle Pain : Neck pain 
 Pairs = Ankle Pain : Shoulder pain 
 Pairs = Bilateral Mastectomy : Breast cancer 
 Pairs = Bilateral Mastectomy : Complex regional pain syndrome 
 Pairs = Breast cancer : Complex regional pain syndrome 
 Pairs = Crohn's disease : Rectovaginal fistula 
 Pairs = Hip Pain : Neck pain 
 Pairs = Neck pain : Shoulder pain 
 Pairs = Obsessive-compulsive disorder : Ovarian cysts 



# PCA